In [379]:
import requests
from bs4 import BeautifulSoup
import time
import xlsxwriter as xlsxwriter
import csv
import math
import re
from datetime import datetime
import numpy as np
import pandas as pd

In [380]:
pd.set_option('display.max_rows',1000)

In [381]:
df_raw = pd.read_csv("크롤링_total.csv", index_col = False)
df_genne = pd.read_csv("genre.csv", index_col = False)
df_raw = pd.concat([df_raw, df_genne])
df = df_raw
df

,영화명,개봉일,매출액,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,배급사,등급,장르,감독,배우,Unnamed: 0
0,태극기 휘날리며,2004-02-05,15687180500,15687180500,2544911,2544911,115,22375,한국,"(주)다자인소프트,(주)쇼박스",15세관람가,"전쟁,드라마",강제규,"장동건,원빈,이은주,공형진,조원희,이장욱,이영란,정희준,박유신,김효열,김효준,최재혁...",NaN
1,트로이,2004-05-21,12777346000,12777346000,2001293,2001293,177,21245,미국,워너브러더스 코리아(주),15세관람가,사극,볼프강 페터젠,"다이앤 크루거,올랜도 블룸,에릭 바나",NaN
2,내 머리 속의 지우개,2004-11-05,11919939500,11919939500,1885827,1885827,206,29532,한국,CJ ENM,12세관람가,멜로/로맨스,이재한,"정우성,손예진,데이비드 맥기니스,김부선,정광재,최기호,박경진,장재용,전은희,이영필,...",NaN
3,귀신이 산다,2004-09-17,11898748500,11898748500,1875936,1875936,174,27936,한국,(주)시네마서비스,12세관람가,"코미디,공포(호러),판타지",김상진,"김응수,장서희,차승원,김서현,최기웅,유대현,김성미,태인호,임상란,장승현,김정우,김양...",NaN
4,투모로우,2004-06-03,11658584500,11658584500,1830767,1830767,164,22881,미국,이십세기폭스필름코퍼레이션,12세관람가,SF,롤랜드 에머리히,"데니스 퀘이드,제이크 질렌할,이안 홈",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
670,너무 많이 본 사나이,NaN,80000,80000,8,8,1,1,한국,NaN,NaN,장르없음,손재곤,NaN,670.0
671,파마하는 날,NaN,3200,3200,4,4,1,1,한국,NaN,NaN,장르없음,송상현,NaN,671.0
672,오리엔테이션,NaN,267500,267500,214,214,1,4,한국,NaN,NaN,장르없음,송윤희,NaN,672.0
673,태양의 길목,NaN,85000,85000,17,17,1,2,중국,NaN,NaN,장르없음,송태가,NaN,673.0


In [382]:
df = df.loc[:, "영화명": "배우"]
df

,영화명,개봉일,매출액,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,배급사,등급,장르,감독,배우
0,태극기 휘날리며,2004-02-05,15687180500,15687180500,2544911,2544911,115,22375,한국,"(주)다자인소프트,(주)쇼박스",15세관람가,"전쟁,드라마",강제규,"장동건,원빈,이은주,공형진,조원희,이장욱,이영란,정희준,박유신,김효열,김효준,최재혁..."
1,트로이,2004-05-21,12777346000,12777346000,2001293,2001293,177,21245,미국,워너브러더스 코리아(주),15세관람가,사극,볼프강 페터젠,"다이앤 크루거,올랜도 블룸,에릭 바나"
2,내 머리 속의 지우개,2004-11-05,11919939500,11919939500,1885827,1885827,206,29532,한국,CJ ENM,12세관람가,멜로/로맨스,이재한,"정우성,손예진,데이비드 맥기니스,김부선,정광재,최기호,박경진,장재용,전은희,이영필,..."
3,귀신이 산다,2004-09-17,11898748500,11898748500,1875936,1875936,174,27936,한국,(주)시네마서비스,12세관람가,"코미디,공포(호러),판타지",김상진,"김응수,장서희,차승원,김서현,최기웅,유대현,김성미,태인호,임상란,장승현,김정우,김양..."
4,투모로우,2004-06-03,11658584500,11658584500,1830767,1830767,164,22881,미국,이십세기폭스필름코퍼레이션,12세관람가,SF,롤랜드 에머리히,"데니스 퀘이드,제이크 질렌할,이안 홈"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
670,너무 많이 본 사나이,NaN,80000,80000,8,8,1,1,한국,NaN,NaN,장르없음,손재곤,NaN
671,파마하는 날,NaN,3200,3200,4,4,1,1,한국,NaN,NaN,장르없음,송상현,NaN
672,오리엔테이션,NaN,267500,267500,214,214,1,4,한국,NaN,NaN,장르없음,송윤희,NaN
673,태양의 길목,NaN,85000,85000,17,17,1,2,중국,NaN,NaN,장르없음,송태가,NaN


### 2. 중복값 제거

In [383]:
df = df.sort_values(["감독","누적관객수","누적매출액"], ascending=True)

In [384]:
df.drop_duplicates(['영화명',"감독"], keep = 'last',inplace=True)

In [385]:
df["영화명"].value_counts()

로미오와 줄리엣          6
카르멘               6
토스카               6
욕망                5
돈                 4
                 ..
아내의 동창회 2         1
온천에서 유혹하는 처제      1
매혹의 스캔들           1
굿 윌 헌팅            1
일본꽐라녀:달아오른 몸뚱이    1
Name: 영화명, Length: 20749, dtype: int64

### 3. 장르 전처리

In [386]:
df["장르"].value_counts()

드라마                              4336
멜로/로맨스                           2038
다큐멘터리                            2001
애니메이션                            1811
액션                               1453
장르없음                             1282
코미디                              1026
에로                                943
성인물(에로)                           759
공포(호러)                            549
스릴러                               462
범죄                                366
SF                                243
미스터리                              161
판타지                               132
공포                                132
공연                                129
전쟁                                126
가족                                 95
어드벤처                               82
모험                                 79
뮤지컬                                61
드라마,멜로/로맨스                         57
멜로/로맨스,코미디                         51
코미디,드라마                            47
서부                                 45
사극          

In [387]:
## nan값 확인 후 처리
df["장르"].isnull().sum() #결측값 2396

1844

In [388]:
df.tail()

,영화명,개봉일,매출액,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,배급사,등급,장르,감독,배우
20309,댄싱 인 자파,NaN,88000,88000,15,15,1,2,미국,NaN,NaN,다큐멘터리,힐라 메달리아,NaN
23238,위기의 30대 여자들,NaN,946400,946400,136,136,1,2,이스라엘,NaN,NaN,다큐멘터리,힐라 메달리아,NaN
20310,16세의 사운드트랙,2020-07-02,3543400,3543400,413,413,19,150,영국,(주)마노엔터테인먼트,12세이상관람가,코미디,힐러리 셰익스피어,"제임스 캘로웨이,스칼렛 마셜"
2785,나의 말을 지켜봐: 어느 귀 먼 연예인들의 이야기,NaN,293500,293500,114,114,1,2,미국,NaN,NaN,다큐멘터리,힐러리 스칼,NaN
20311,분노의 건샷,2016-12-14,4000,4000,1,1,1,1,미국,(주) 케이알씨지,[국내] 15세 관람가,범죄,힐러리안 뱅크스,"제임스 와일더,조아큄 드 알메이다,레베카 메이 파머"


In [389]:
df.reset_index(inplace=True)

In [390]:
for i in range(len(df["장르"])):
    if type(df["장르"][i]) == float:
        df["장르"][i] = "장르없음"
    else:
        df["장르"][i] = df["장르"][i].split(",")[0]

<ipython-input-390-ce42fc033b6c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["장르"][i] = df["장르"][i].split(",")[0]
<ipython-input-390-ce42fc033b6c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["장르"][i] = "장르없음"


In [391]:
# 카테고리 만들기

df.replace("성인물(에로)", "에로", inplace = True)
df.replace("서부극(웨스턴)", "서부", inplace = True)
df.replace("공연실황", "공연", inplace = True)
df.replace("서스펜스", "스릴러", inplace = True)
df.replace("공포(호러)", "공포", inplace = True)
df.replace("멜로/로맨스", "멜로로맨스", inplace = True)
df.replace("실험", "다큐멘터리", inplace = True)

In [392]:
#기타장르 제거
guitar_idx = df[df["장르"]=="기타"].index
df.drop(guitar_idx, inplace=True)

In [393]:
df["장르"].value_counts()

드라마      4552
장르없음     3126
멜로로맨스    2167
다큐멘터리    2014
애니메이션    1899
에로       1711
액션       1586
코미디      1173
공포        756
스릴러       510
범죄        410
SF        294
미스터리      180
공연        172
판타지       149
전쟁        132
가족        117
어드벤처      102
모험         79
뮤지컬        68
서부         56
사극         53
느와르         9
무협          8
서사          2
Name: 장르, dtype: int64

### 4. 매출액  / 누적매출액 : 음수 -> 양수

In [394]:
values = []

for value in df['매출액'].values:
    value = abs(value)
    values.append(value)

In [395]:
stacked_values = []

for value in df['누적매출액'].values:
    value = abs(value)
    stacked_values.append(value)

In [396]:
# df['매출액'] = values
# df['누적매출액'] = stacked_values

### 5. 등급 전처리

In [397]:
# 등급에서 "[국내] " 없애기
df['등급'].value_counts()

#df.replace("성인물(에로)", "에로", inplace = True)
df.replace([" 청소년 관람불가 ","청소년 관람불가 ","청소년 관람불가","[국내] 청소년 관람불가","고등학생이상관람가,청소년관람불가","청소년관람불가,12세관람가","국민학생관람불가,청소년관람불가","청소년관람불가,고등학생이상관람가","18세관람가,청소년관람불가","18세 미만인 자는 관람할 수 없는 등급","미성년자관람불가",'18세관람가', "연소자관람불가","연소자관람불가,청소년관람불가"], '청소년관람불가',inplace=True)
df.replace([" 15세 관람가 ","15세 관람가 ","[국내] 15세 관람가"," 15세 관람가","15세관람가","고등학생이상관람가","고등학생이상관람가,15세이상관람가","15세 미만인 자는 관람할 수 없는 등급","15세관람가,15세이상관람가","15세이상관람가,중학생이상관람가","15세이상관람가,미성년자관람불가","청소년관람불가,15세이상관람가","연소자관람불가,15세이상관람가","국민학생관람불가,15세이상관람가","15세이상관람가,전체관람가","15세이상관람가,18세 미만인 자는 관람할 수 없는 등급","연소자관람가,15세이상관람가","18세관람가,15세이상관람가","15세 미만인 자는 관람할 수 없는 등급 ,15세이상관람가"], '15세이상관람가',inplace=True)
df.replace([" 12세 관람가 ","12세 관람가 "," 12세 관람가","[국내] 12세 관람가","12세관람가","중학생이상관람가","국민학생관람불가","12세이상관람가,12세관람가","12세 미만인 자는 관람할 수 없는 등급","12세이상관람가,중학생이상관람가","12세이상관람가,국민학생관람불가","12세이상관람가,전체관람가","12세이상관람가,15세 미만인 자는 관람할 수 없는 등급","12세이상관람가,15세이상관람가","12세이상관람가,고등학생이상관람가","12세이상관람가,국민학생관람불가,15세이상관람가","12세이상관람가,연소자관람가","15세관람가,12세이상관람가"],"12세이상관람가", inplace=True)
df.replace(["전체관람가,중학생이상관람가"," 전체 관람가 ","전체 관람가 ","[국내] 전체 관람가","연소자관람가","모든 관람객이 관람할 수 있는 등급","연소자관람가,전체관람가","청소년관람불가,전체관람가","미성년자관람가","모든 관람객이 관람할 수 있는 등급,전체관람가","12세이상관람가,연소자관람가,전체관람가"],"전체관람가",inplace=True)

In [398]:
df["등급"].value_counts()

청소년관람불가     6316
15세이상관람가    3887
12세이상관람가    2173
전체관람가       1889
Name: 등급, dtype: int64

### 6. 감독, 배우, 배급사

In [399]:
# 감독, 배우, 배급사 결측값 0으로 처리

df['배우'] = df['배우'].fillna(0)
df['감독'] = df['감독'].fillna(0)
df['배급사'] = df['배급사'].fillna(0)
df['등급'] = df['등급'].fillna("등급없음")

In [407]:
del df['index']

In [408]:
df

,영화명,개봉일,매출액,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,배급사,등급,장르,감독,배우
0,"416 프로젝트 ""공동의 기억: 트라우마""",NaN,2192500,2192500,437,437,3,17,한국,0,등급없음,다큐멘터리,416연대 미디어위원회,0
1,미씨카: 더 네크로맨서,2017-11-02,300000,300000,60,60,1,1,미국,(주)제이에이와이이엔터테인먼트,15세이상관람가,판타지,A. 토드 스미스,"애덤 존슨,로버트 제인,케빈 소르보"
2,월드 워 핵전쟁의 서막,2019-03-07,26000,26000,17,17,5,17,뉴질랜드,(주)풍경소리,15세이상관람가,액션,A.K. 스트롬,"모건 브래들리,그레이엄 빈센트,캠벨 루셀"
3,울프라이징,2016-11-17,300000,300000,60,60,1,1,미국,주식회사 루믹스미디어,15세이상관람가,공포,BC 퍼트니,빌 오버스트 주니어
4,제3의앵글 : 형형색색대만전,NaN,1830000,1830000,305,305,1,2,대만,0,등급없음,장르없음,BITO,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21320,댄싱 인 자파,NaN,88000,88000,15,15,1,2,미국,0,등급없음,다큐멘터리,힐라 메달리아,0
21321,위기의 30대 여자들,NaN,946400,946400,136,136,1,2,이스라엘,0,등급없음,다큐멘터리,힐라 메달리아,0
21322,16세의 사운드트랙,2020-07-02,3543400,3543400,413,413,19,150,영국,(주)마노엔터테인먼트,12세이상관람가,코미디,힐러리 셰익스피어,"제임스 캘로웨이,스칼렛 마셜"
21323,나의 말을 지켜봐: 어느 귀 먼 연예인들의 이야기,NaN,293500,293500,114,114,1,2,미국,0,등급없음,다큐멘터리,힐러리 스칼,0


### 7. 대표국적

In [401]:
nation = ["미국", "한국", "일본", "프랑스", "영국", "독일", "중국", "이탈리아", "캐나다", "스페인"]

for i in range(len(df)):
    if df.iloc[i]["대표국적"] not in nation:
        df.replace(df.iloc[i]["대표국적"],"기타",inplace=True)

In [402]:
# nation = ["미국", "한국", "일본", "프랑스", "영국", "독일", "중국", "이탈리아", "캐나다", "스페인"]

# for i in df["대표국적"]:
#     if i not in nation:
#         i.replace(i,"기타",inplace=True)

In [403]:
df.head(100)

,index,영화명,개봉일,매출액,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,배급사,등급,장르,감독,배우
0,23682,"416 프로젝트 ""공동의 기억: 트라우마""",NaN,2192500,2192500,437,437,3,17,한국,0,등급없음,다큐멘터리,416연대 미디어위원회,0
1,22580,미씨카: 더 네크로맨서,2017-11-02,300000,300000,60,60,1,1,미국,(주)제이에이와이이엔터테인먼트,15세이상관람가,판타지,A. 토드 스미스,"애덤 존슨,로버트 제인,케빈 소르보"
2,3092,월드 워 핵전쟁의 서막,2019-03-07,26000,26000,17,17,5,17,뉴질랜드,(주)풍경소리,15세이상관람가,액션,A.K. 스트롬,"모건 브래들리,그레이엄 빈센트,캠벨 루셀"
3,3093,울프라이징,2016-11-17,300000,300000,60,60,1,1,미국,주식회사 루믹스미디어,15세이상관람가,공포,BC 퍼트니,빌 오버스트 주니어
4,3094,제3의앵글 : 형형색색대만전,NaN,1830000,1830000,305,305,1,2,대만,0,등급없음,장르없음,BITO,0
5,21171,연쇄살인마의 이중생활,2020-06-18,16000,16000,7,7,5,7,스페인,주식회사 마운틴픽쳐스,15세이상관람가,범죄,C. 마틴 페레라,"안나 알렌,나초 프레스네다,메르세데스 카스트로"
6,20514,몬스터 아일랜드,2017-10-27,3000,3000,1,1,1,1,미국,주식회사 레드아이스,15세이상관람가,어드벤처,C. 블레이크 에번든,존 W. 바우어스
7,3095,잃어버린 세계를 찾아 2015,2015-12-03,4000,4000,1,1,1,1,미국,(주)케이알씨지,12세이상관람가,어드벤처,C. 토마스 하우웰,"C. 토마스 하우웰,티모시 바톰즈"
8,3096,과부: 탐스러운 육체,2016-11-19,5000,5000,1,1,1,1,일본,(주)영진크리에이티브,청소년관람불가,멜로로맨스,C.나가후쿠,"타치바나 케이코,이치죠 리노"
9,3097,페이스트리의 왕,NaN,642000,642000,107,107,1,2,미국,0,등급없음,장르없음,"D.A. 페네베이커,크리스 헤지더스",0


### 중간 저장

In [404]:
df.to_csv("중간저장.csv" ,encoding = 'utf-8-sig')

In [405]:
movie = pd.read_csv("중간저장.csv", index_col = False, thousands = ',')
movie = movie.loc[:, "영화명" : "배우"]
movie

,영화명,개봉일,매출액,누적매출액,관객수,누적관객수,스크린수,상영횟수,대표국적,배급사,등급,장르,감독,배우
0,"416 프로젝트 ""공동의 기억: 트라우마""",NaN,2192500,2192500,437,437,3,17,한국,0,등급없음,다큐멘터리,416연대 미디어위원회,0
1,미씨카: 더 네크로맨서,2017-11-02,300000,300000,60,60,1,1,미국,(주)제이에이와이이엔터테인먼트,15세이상관람가,판타지,A. 토드 스미스,"애덤 존슨,로버트 제인,케빈 소르보"
2,월드 워 핵전쟁의 서막,2019-03-07,26000,26000,17,17,5,17,뉴질랜드,(주)풍경소리,15세이상관람가,액션,A.K. 스트롬,"모건 브래들리,그레이엄 빈센트,캠벨 루셀"
3,울프라이징,2016-11-17,300000,300000,60,60,1,1,미국,주식회사 루믹스미디어,15세이상관람가,공포,BC 퍼트니,빌 오버스트 주니어
4,제3의앵글 : 형형색색대만전,NaN,1830000,1830000,305,305,1,2,대만,0,등급없음,장르없음,BITO,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21320,댄싱 인 자파,NaN,88000,88000,15,15,1,2,미국,0,등급없음,다큐멘터리,힐라 메달리아,0
21321,위기의 30대 여자들,NaN,946400,946400,136,136,1,2,이스라엘,0,등급없음,다큐멘터리,힐라 메달리아,0
21322,16세의 사운드트랙,2020-07-02,3543400,3543400,413,413,19,150,영국,(주)마노엔터테인먼트,12세이상관람가,코미디,힐러리 셰익스피어,"제임스 캘로웨이,스칼렛 마셜"
21323,나의 말을 지켜봐: 어느 귀 먼 연예인들의 이야기,NaN,293500,293500,114,114,1,2,미국,0,등급없음,다큐멘터리,힐러리 스칼,0


### 8. 배급사

In [406]:
# 배급사 1명만 남기기

# for i in range(len(df["배급사"])):
#     if type(df["배급사"][i]) == float:
#         df["배급사"][i] = 0
#     else:
#         df["배급사"][i] = df["배급사"][i].split(",")[0]